In [ ]:

from src.data.dataset_reader import DatasetReader
from pathlib import Path
%load_ext autoreload
%autoreload 2
# ignore warnings
import warnings
warnings.filterwarnings('ignore')
data_folder = Path("../data/raw").resolve()

amazon = DatasetReader(data_folder).get_data_csv('Amazon Sale Report.csv')
#int_sales = DatasetReader(data_folder).get_data_csv('International sale Report.csv')
#p_l = DatasetReader(data_folder).get_data_csv('P LMarch2021.csv')
#may_2022 = DatasetReader(data_folder).get_data_csv('May-2022.csv')
#sale_report = DatasetReader(data_folder).get_data_csv('Sale Report.csv')


This dataset provides an in-depth look at the profitability of e-commerce sales. It contains data on a variety of sales channels, including Shiprocket and INCREFF, as well as financial information on related expenses and profits. The columns contain data such as SKU codes, design numbers, stock levels, product categories, sizes and colors. In addition to this we have included the MRPs across multiple stores like Ajio MRP , Amazon MRP , Amazon FBA MRP , Flipkart MRP , Limeroad MRP Myntra MRP and PaytmMRP along with other key parameters like amount paid by customer for the purchase , rate per piece for every individual transaction Also we have added transactional parameters like Date of sale months category fulfilledby B2b Status Qty Currency Gross amt . This is a must-have dataset for anyone trying to uncover the profitability of e-commerce sales in today's marketplace.
This dataset provides a comprehensive overview of e-commerce sales data from different channels covering a variety of products. Using this dataset, retailers and digital marketers can measure the performance of their campaigns more accurately and efficiently.

The following steps help users make the most out of this dataset:

Analyze the general sales trends by examining info such as month, category, currency, stock level, and customer for each sale. This will give you an idea about how your e-commerce business is performing in each channel.

Review the Shiprocket and INCREF data to compare and analyze profitability via different fulfilment methods. This comparison would enable you to make better decisions towards maximizing profit while minimizing costs associated with each method’s referral fees and fulfillment rates.

Compare prices between various channels such as Amazon FBA MRP, Myntra MRP, Ajio MRP etc using  the corresponding columns for each store (Amazon MRP etc). You can judge which stores are offering more profitable margins without compromising on quality by analyzing these pricing points in combination with other information related to product sales (TP1/TP2 - cost per piece).

Look at customer specific data such as TP 1/TP 2 combination wise Gross Amount or Rate info in terms price per piece or total gross amount generated by any SKU dispersed over multiple customers with relevant dates associated to track individual item performance relative to others within its category over time periods shortlisted/filtered appropriately.. Have an eye on items commonly utilized against offers or promotional discounts offered hence crafting strategies towards inventory optimization leading up-selling operations.?

Finally Use Overall ‘Stock’ details along all the P & L Data including Yearly Expenses_IIGF information record for takeaways which might be aimed towards essential cost cutting measures like switching amongst delivery options carefully chosen out of Shiprocket & INCREFF leadings away from manual inspections catering savings under support personnel outsourcing structures.?  

By employing a comprehensive understanding on how our internal subsidiaries perform globally unless attached respective audits may provide us remarkably lower operational costs servicing confidence; costing far lesser than being incurred taking into account entire pallet shipments tracking sheets representing current level supply chains efficiencies achieved internally., then one may finally scale profits exponentially increases cut down unseen losses followed up introducing newer marketing campaigns necessarily tailored according playing around multiple goods based spectrums due powerful backing suitable transportation boundaries set carefully

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
amazon.head(3)
#amazon.query('SKU == "SET389-KR-NP-S"').head(1)
                                             


In [ ]:
# Create a new feature called sales that is the  the count of "Amount" grouped by "SKU" in the amazon dataset
#amazon['sales'] = amazon.groupby('SKU')['Amount'].transform('count')
#amazon.drop(columns=[ 'Order ID', 'ASIN', 'Unnamed: 22', 'index'], inplace=True
amazon['Date'] = pd.to_datetime(amazon['Date'])
amazon['ship-postal-code'] = amazon['ship-postal-code'].astype('object')
amazon_beta =  amazon[['Order ID','Date', 'SKU', 'Qty', 'Amount']]

In [ ]:
amazon_beta =  amazon[['Order ID','Date', 'SKU', 'Qty', 'Amount']]

In [ ]:
from src.data.eda import Eda
eda = Eda(    
)

eda.missing_values_table(amazon_beta)

In [ ]:
amazon_beta.rename(columns={'Amount': 'price'}, inplace=True)

In [ ]:
amazon_beta.head()
# Create a column called "sales" that is the count of the "Order ID" grouped by "SKU" and "Date" in the amazon_beta dataset
amazon_beta['sales'] = amazon_beta.groupby(['SKU', 'Date'])['Order ID'].transform('count')


In [ ]:
amazon_beta.query('SKU == "SET389-KR-NP-S"').sort_values(by=['Date'], ascending=True).head()

In [ ]:
# show duplicates in amazon_beta 
#amazon_beta[amazon_beta.duplicated(keep=False)].sort_values(by='SKU', ascending=True).head(10)
# drop duplicates in amazon_beta
#amazon_beta.drop_duplicates(inplace=True)

In [ ]:
eda.missing_values_table(amazon_beta)

In [ ]:
amazon_beta.sort_values(by=['SKU', 'Date'], ascending=True, inplace=True)

In [ ]:
from src.data.feature_engineering import FeatureEngineeringProcess
fep = FeatureEngineeringProcess()

In [ ]:
amazon_beta_fe1 = fep.price_sales_correlation_features_updated(amazon_beta, 15, [1, 2], [1,2],)
amazon_beta_fe1.head(3)

In [ ]:
#print this instruction: amazon_beta_fe1.query('SKU == "SET389-KR-NP-S"') without index
amazon_beta_fe1.query('SKU == "SET389-KR-NP-S"').head(3)

In [ ]:
amazon_beta_fe2 = fep.normalize_features(amazon_beta_fe1, 8, 15)

In [ ]:
amazon_beta_fe2.query('SKU == "SET389-KR-NP-S"').T

In [ ]:
amazon_beta_fe3 = fep.filter_stability_periods(amazon_beta_fe2, 7, 0.04)

In [ ]:
# print without index amazon_beta_fe3.query('SKU == "SET389-KR-NP-S"') without index column


amazon_beta_fe3[['Order ID', 'Date', 'SKU', 'price', 'sales','avg_price_last_7_days', 'price_variation']].query('SKU == "SET389-KR-NP-S"')


# NOTE: Still seeing repeated values in the "price" column. Either low the threshold or increase removal_percentage


In [ ]:

eda.missing_values_table(amazon_beta_fe3)

In [ ]:
amazon_beta_fe3.head()

In [ ]:
# drop rows where price is null
amazon_beta_train = amazon_beta_fe3.dropna(subset=['price'])

In [ ]:
amazon_beta_train.fillna(0, inplace=True)

In [ ]:
amazon_beta_train1= fep.datetime_transform(amazon_beta_train, 'Date',['day_of_the_year', 'month', 'day_of_the_week', 'season'])

In [ ]:
# Drop Order Id, price_variation, insufficient_data
amazon_beta_train1.drop(columns=['Order ID', 'price_variation', 'insufficient_data'], inplace=True)

In [ ]:
amazon_beta_train1.head()

In [ ]:
from src.data.data_splitter import DataSplitter

splitter = DataSplitter(amazon_beta_train1, train_months=3, test_weeks=2)
train, test = splitter.split_data()

In [ ]:
# Drop Date column
train.drop(columns=['Date'], inplace=True)
test.drop(columns=['Date'], inplace=True)
# one hot encode Date_season
train = pd.get_dummies(train, columns=['Date_season'])
test = pd.get_dummies(test, columns=['Date_season'])

In [ ]:
train.info()

In [ ]:

from xgboost import XGBRegressor



# Define and train XGBoost model
xgboost_model = XGBRegressor(n_estimators=500, learning_rate=0.1, max_depth=6)
xgboost_model.fit(train.drop(['SKU', 'sales'], axis=1), train['sales'])

# Group test data by SKU
grouped = test.groupby('SKU')

# Perform inference for each SKU in test data
for sku, group_data in grouped:
    print(f"Performing inference for SKU: {sku}")
    
    # Select one row from group data (one set of feature values for this SKU)
    feature_values = group_data.iloc[0]
    
    # Set the base price from data
    base_price = feature_values['price']
    
    # Define the price points
    price_points = [base_price, 
                    base_price * 0.9,   # base_price - 10%
                    base_price * 0.85,  # base_price - 15%
                    base_price * 1.1,   # base_price + 10%
                    base_price * 1.15]  # base_price + 15%
    
    predicted_sales_xgboost = []
    
    # For each price point, use the trained model to predict sales
    for price in price_points:
        # Change the price while keeping other features constant
        test_features = feature_values.copy()
        test_features['price'] = price
        
        # Remove the SKU field to match the input shape expected by the model
        test_features = test_features.drop(['SKU', 'sales'])

        # Predict sales using the XGBoost model
        predicted_sales_xgboost.append(xgboost_model.predict(test_features.values.reshape(1, -1))[0])
    
    # Print the predicted sales at each price point
    for price, xgboost_sales in zip(price_points, predicted_sales_xgboost):
        print(f"Price: {price}, XGBoost Predicted Sales: {xgboost_sales}")

    print("\n")


In [ ]:
train